In [1]:
import pandas as pd
import sqlalchemy
import requests
from bs4 import BeautifulSoup
from splinter import Browser
import time

executable_path = {'executable_path': 'chromedriver.exe'}

In [27]:
get_url = 'https://www.mlb.com/'
html = requests.get(get_url)
soup = BeautifulSoup(html.text, 'html.parser')
#soup.prettify()

# top headlines on mlb.com

In [28]:
top_headlines = []
results = soup.select('div.l-grid div.p-headline-stack')
for result in results[3].select('li.p-headline-stack__headline'):
#     print(result.text)
#     print(result.a['href'])
#     print('-'*12)
    top_headlines.append({'headline':result.text, 'headline_url':result.a['href']})
print(top_headlines)

[{'headline': "Making sense of Bumgarner's no-trade list", 'headline_url': 'https://www.mlb.com/news/madison-bumgarner-no-trade-list'}, {'headline': 'Shoulder strain keeping Stanton sidelined', 'headline_url': 'https://www.mlb.com/news/giancarlo-stanton-injured-shoulder-strain'}, {'headline': 'Paddack-Kershaw comparison favors the rookie', 'headline_url': 'https://www.mlb.com/news/clayton-kershaw-chris-paddack-face-off'}, {'headline': '9 relievers who could be dealt this summer', 'headline_url': 'https://www.mlb.com/news/relief-pitchers-who-could-be-traded'}, {'headline': '1st Cy Young poll has some surprises at top', 'headline_url': 'https://www.mlb.com/news/2019-cy-young-award-poll-leaders'}, {'headline': "He's a No. 1 overall pick ... and playing like it", 'headline_url': 'https://www.mlb.com/news/dansby-swanson-is-fulfilling-his-potential'}, {'headline': "What to expect from Brewers' Hiura in bigs", 'headline_url': 'https://www.mlb.com/news/brewers-top-prospect-keston-hiura-called-

# parse top headline articles and pull player names and IDs and add to prior dictionary

In [64]:
browser = Browser('chrome', **executable_path, headless=False)
i = 0
for each_article in top_headlines:
    get_url = each_article['headline_url']
#     html = requests.get(get_url)
#     soup = BeautifulSoup(html.text, 'html.parser')
# had to use chromedriver because for some articles, the player names were linked in <a href> but others,
# the request would get <forge-entity> that would be converted to <a> after it had already requests.get-ed
    
    browser.visit(get_url)
    #time.sleep(2)
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
    
    article_info = []
    results = soup.select('div.template-article__content div.article-item__body')
    articlebody_results = results[0].select('p a')
    for each_name in articlebody_results:
        #print(f"{each_name.text} with player ID {each_name['slug'][-6:]}")
        if each_name['href'][:27] == 'https://www.mlb.com/player/':
            article_info.append({'player_name':each_name.text, 'player_id':each_name['href'][-6:]})
    #print(article_info)
    top_headlines[i]['article_names'] = article_info
    i += 1
browser.quit()
print(top_headlines)

[{'headline': "Making sense of Bumgarner's no-trade list", 'headline_url': 'https://www.mlb.com/news/madison-bumgarner-no-trade-list', 'article_names': [{'player_name': 'Madison Bumgarner', 'player_id': '518516'}]}, {'headline': 'Shoulder strain keeping Stanton sidelined', 'headline_url': 'https://www.mlb.com/news/giancarlo-stanton-injured-shoulder-strain', 'article_names': [{'player_name': 'Giancarlo Stanton', 'player_id': '519317'}]}, {'headline': 'Paddack-Kershaw comparison favors the rookie', 'headline_url': 'https://www.mlb.com/news/clayton-kershaw-chris-paddack-face-off', 'article_names': [{'player_name': 'Chris Paddack', 'player_id': '663978'}, {'player_name': 'Clayton Kershaw', 'player_id': '477132'}]}, {'headline': '9 relievers who could be dealt this summer', 'headline_url': 'https://www.mlb.com/news/relief-pitchers-who-could-be-traded', 'article_names': [{'player_name': 'Mychal Givens', 'player_id': '571710'}, {'player_name': 'Ian Kennedy', 'player_id': '453178'}, {'player_n

# top most rockies headline, URL, referenced player names and IDs

In [73]:
get_url = 'https://www.mlb.com/rockies'
html = requests.get(get_url)
soup = BeautifulSoup(html.text, 'html.parser')
#top_cr_headline = []
results = soup.select('div.l-grid div.p-headline-stack')
top_result = results[3].select('li.p-headline-stack__headline')[0]
top_cr_headline = {'headline':top_result.text.strip(), 'headline_url':top_result.a['href']}
print(top_cr_headline)

{'headline': 'Valaika called up; surgery possible for Anderson', 'headline_url': 'https://www.mlb.com/rockies/news/pat-valaika-called-up-tyler-anderson-knee-decision'}


In [74]:
browser = Browser('chrome', **executable_path, headless=False)
get_url = top_cr_headline['headline_url']
#     html = requests.get(get_url)
#     soup = BeautifulSoup(html.text, 'html.parser')
# had to use chromedriver because for some articles, the player names were linked in <a href> but others,
# the request would get <forge-entity> that would be converted to <a> after it had already requests.get-ed

browser.visit(get_url)
html = browser.html
soup = BeautifulSoup(html,'html.parser')

article_info = []
results = soup.select('div.template-article__content div.article-item__body')
articlebody_results = results[0].select('p a')
for each_name in articlebody_results:
    if each_name['href'][:27] == 'https://www.mlb.com/player/':
        article_info.append({'player_name':each_name.text, 'player_id':each_name['href'][-6:]})
top_cr_headline['article_names'] = article_info
browser.quit()
print(top_cr_headline)

{'headline': 'Valaika called up; surgery possible for Anderson', 'headline_url': 'https://www.mlb.com/rockies/news/pat-valaika-called-up-tyler-anderson-knee-decision', 'article_names': [{'player_name': 'Pat Valaika', 'player_id': '642162'}, {'player_name': 'Garrett Hampson', 'player_id': '641658'}, {'player_name': 'Tyler Anderson', 'player_id': '542881'}]}


# most recent rockies video

In [94]:
cr_team_id = 115
video_search_url = 'https://www.mlb.com/video/search/tag/teamid-'+str(cr_team_id)
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(video_search_url)
html = browser.html
soup = BeautifulSoup(html,'html.parser')

In [102]:
results = soup.select('div.video-preview')
video_page_url = 'https://www.mlb.com' + results[0].select('a')[0]['href']
browser.visit(video_page_url)
time.sleep(5)
html = browser.html
soup2 = BeautifulSoup(html,'html.parser')
video_result = {}
video_wrapper = soup2.select('div#video-section__wrapper meta')
for each_meta in video_wrapper:
    if each_meta['itemprop'] == 'name':
        video_result['name'] = each_meta['content']
    elif each_meta['itemprop'] == "contentURL":
        video_result['url'] = each_meta['content']
    elif each_meta['itemprop'] == "description":
        video_result['desc'] = each_meta['content']
print(video_result)
browser.quit()

{'name': "Blackmon discusses Rockies in '19", 'desc': "Rockies outfielder Charlie Blackmon talks about the club's season so far in 2019", 'url': 'https://cuts.diamond.mlb.com/FORGE/2019/2019-05/14/1016a9a3-763d632f-078c4081-csvm-diamondx64-asset_1280x720_59_4000K.mp4'}


# most recent video featuring hitting and/or pitching leader using ID?

In [103]:
player_id = 518516 #using madison bumgarner right now, can do a list and throw this all in a for loop

video_search_url = 'https://www.mlb.com/video/search/tag/playerid-'+str(player_id)

browser = Browser('chrome', **executable_path, headless=False)
browser.visit(video_search_url)
html = browser.html
soup = BeautifulSoup(html,'html.parser')
results = soup.select('div.video-preview')
video_page_url = 'https://www.mlb.com' + results[0].select('a')[0]['href']
browser.visit(video_page_url)
time.sleep(5)
html = browser.html
soup2 = BeautifulSoup(html,'html.parser')
video_result = {}
video_wrapper = soup2.select('div#video-section__wrapper meta')
for each_meta in video_wrapper:
    if each_meta['itemprop'] == 'name':
        video_result['name'] = each_meta['content']
    elif each_meta['itemprop'] == "contentURL":
        video_result['url'] = each_meta['content']
    elif each_meta['itemprop'] == "description":
        video_result['desc'] = each_meta['content']
print(video_result)
browser.quit()

{'name': 'Could Bumgarner be traded?', 'desc': "The guys on MLB Now discuss Madison Bumgarner's no-trade list", 'url': 'https://cuts.diamond.mlb.com/FORGE/2019/2019-05/13/c651f26e-4b1c5fa1-ea3a17aa-csvm-diamondx64-asset_1280x720_59_4000K.mp4'}


# pull standings from mlb.com/standings

In [107]:
browser = Browser('chrome', **executable_path, headless=False)
browser.visit('https://www.mlb.com/standings')
time.sleep(3)
html = browser.html
#tables are created asynchronously so can't just read_html the page
browser.quit()

[  American League EastAL East   W   L    PCT    GB  WCGB  L10 STRK   RS   RA  \
 0               E-Tampa BayTB  24  15  0.615     -     -  5-5   L1  180  123   
 1             E-NY YankeesNYY  24  16  0.600   0.5  +2.5  7-3   W1  200  164   
 2                 E-BostonBOS  22  19  0.537   3.0     -  8-2   W5  220  193   
 3                E-TorontoTOR  16  24  0.400   8.5   5.5  2-8   L2  141  175   
 4              E-BaltimoreBAL  14  26  0.350  10.5   7.5  4-6   W1  159  234   
 
    DIFF  X-W/L   HOME   AWAY >.500      Next Game  
 0    57  26-13  12-10   12-5   8-7     Live @ MIA  
 1    36  24-16   13-9   11-7   6-7  May 15 vs BAL  
 2    27  23-18   11-7  11-12   4-6    Live vs COL  
 3   -34  16-24   8-13   8-11  5-11    May 14 @ SF  
 4   -75  13-27   6-15   8-11  7-17   May 15 @ NYY  ,
   American League EastAL East   W   L    PCT    GB  WCGB  L10 STRK   RS   RA  \
 0               E-Tampa BayTB  24  15  0.615     -     -  5-5   L1  180  123   
 1             E-NY YankeesNYY 

In [118]:
al_e_df = pd.read_html(html)[0]
al_c_df = pd.read_html(html)[2]
al_w_df = pd.read_html(html)[4]
nl_e_df = pd.read_html(html)[6]
nl_c_df = pd.read_html(html)[8]
nl_w_df = pd.read_html(html)[10]

,National League CentralNL Central,W,L,PCT,GB,WCGB,L10,STRK,RS,RA,DIFF,X-W/L,HOME,AWAY,>.500,Next Game
0,C-Chi CubsCHC,24,14,0.632,-,-,8-2,W2,206,152,54,24-14,15-6,9-8,14-10,Live @ CIN
1,C-MilwaukeeMIL,24,19,0.558,2.5,+0.5,7-3,L3,203,201,2,22-21,16-8,8-11,11-13,Live @ PHI
2,C-St. LouisSTL,22,19,0.537,3.5,0.5,2-8,L3,206,186,20,22-19,14-9,8-10,14-15,Live @ ATL
3,C-PittsburghPIT,20,18,0.526,4.0,1.0,6-4,L1,141,182,-41,15-23,9-9,11-9,4-13,May 14 @ ARI
4,C-CincinnatiCIN,18,23,0.439,7.5,4.5,5-5,L1,171,141,30,24-17,9-8,9-15,8-16,Live vs CHC
